In [3]:
import os
import getpass
# load the environment variables from .env file
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in SF")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

[{'title': 'Friday, April 25, 2025. San Francisco, CA - Weather Forecast', 'url': 'https://weathershogun.com/weather/usa/ca/san-francisco/480/april/2025-04-25', 'content': 'San Francisco, California Weather: Friday, April 25, 2025. Cloudy weather, overcast skies with clouds. Day 61°. Night 48°. Precipitation 6 %.', 'score': 0.9459663}, {'title': 'Weather in San Francisco', 'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1745615351, 'localtime': '2025-04-25 14:09'}, 'current': {'last_updated_epoch': 1745614800, 'last_updated': '2025-04-25 14:00', 'temp_c': 11.1, 'temp_f': 52.0, 'is_day': 1, 'condition': {'text': 'Light rain', 'icon': '//cdn.weatherapi.com/weather/64x64/day/296.png', 'code': 1183}, 'wind_mph': 11.0, 'wind_kph': 17.6, 'wind_degree': 240, 'wind_dir': 'WSW', 'pressure_mb': 1016.0, 'pressur

In [5]:
if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

from langchain.chat_models import init_chat_model

model = init_chat_model("gemini-2.0-flash", model_provider="google_genai")

In [8]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="hi!")])
response.content

'Hi there! How can I help you today?'

In [9]:
model_with_tools = model.bind_tools(tools)

In [10]:
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hi there! How can I help you today?
ToolCalls: []


In [11]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in SF?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': '8d02c271-c99a-4692-abd5-3c20e790f733', 'type': 'tool_call'}]


### this is where the search agent will be created.

In [12]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

In [13]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

[HumanMessage(content='hi!', additional_kwargs={}, response_metadata={}, id='ab94ae28-7ce9-4096-be98-ea0791865bcf'),
 AIMessage(content='Hi there! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run-761176c4-b35c-4722-b4a3-f9a150fbc8dd-0', usage_metadata={'input_tokens': 49, 'output_tokens': 11, 'total_tokens': 60, 'input_token_details': {'cache_read': 0}})]

In [14]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
)
response["messages"]

[HumanMessage(content='whats the weather in sf?', additional_kwargs={}, response_metadata={}, id='ff1cd21d-7892-4a00-a1df-66ea45ef6132'),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': '{"query": "weather in San Francisco"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run-65809f8c-59ca-4d1a-a328-197860d53dfe-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': '11f2c7be-b495-40eb-bc24-686afe3da612', 'type': 'tool_call'}], usage_metadata={'input_tokens': 53, 'output_tokens': 13, 'total_tokens': 66, 'input_token_details': {'cache_read': 0}}),
 ToolMessage(content='[{"title": "Weather in San Francisco", "url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'San Francisco\', \'region\': \'California\', \'country\': \'United St

In [15]:
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

whats the weather in sf?
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (d66c0569-2223-4e8c-a3f8-c897b36bd9b0)
 Call ID: d66c0569-2223-4e8c-a3f8-c897b36bd9b0
  Args:
    query: weather in San Francisco
================================= Tool Message =================================
Name: tavily_search_results_json

[{"title": "Weather in San Francisco", "url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1745613476, 'localtime': '2025-04-25 13:37'}, 'current': {'last_updated_epoch': 1745613000, 'last_updated': '2025-04-25 13:30', 'temp_c': 11.1, 'temp_f': 52.0, 'is_day': 1, 'condition': {'text': 'Light rain', 'icon': '//cdn.weatherapi.com/weather/64x6

In [ ]:
for step, metadata in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]},
    stream_mode="messages",
):
    if metadata["langgraph_node"] == "agent" and (text := step.text()):
        print(text, end="")

The weather in San| Francisco on Friday, April 25, 2025 is light rain. The| temperature is 11.1 degrees Celsius (52.0 degrees Fahrenheit),| but it feels like 9.1 degrees Celsius (48.5 degrees Fahrenheit). The wind is from the WSW at 16.2 k|ph (10.1 mph). The humidity is 83%.|